In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import glob
from pyadigraph import Adigraph
from networkx.algorithms import bipartite
from collections import OrderedDict


class Matching(list):
    def __init__(self, G: nx.Graph):
        """Determine a maximam matching from the given bipartite graph `G`.
            G:nx.Graph, the graph from which to determine the matching.
        """
        self._original_G = G
        self._G = G.copy()
    
    def run(self):
        for node in self._original_G.node:
            if self._is_exposed(node):
                if self._search(node):
                    self._augment()
                else:
                    self._delete(node)
    
    def _delete(self, node: int):
        """Remove given node from graph.
            node:int, the node to remove.
        """
        self._G.remove_node(node)

    def _reverse_path_walker(self, path: list):
        """Lazily yields path edges, from end to start."""
        start, end = path[0][0], path[-1][1]
        previous_node = end
        for (i, j) in reversed(path):
            if previous_node == j:
                yield (i, j)
                previous_node = i
            if previous_node == start:
                break

    def _augment(self):
        """Augments matching folloing augmenting path"""
        for i, (j, k) in enumerate(self._reverse_path_walker(self._path)):
            if i % 2 == 0:
                self.append((j, k))
                self._augment_add_step(j,k)
            else:
                for a, e in enumerate(self):
                    if set(e) == set((j, k)):
                        del self[a]
                        break
                self._augment_remove_step(j,k)
            

    def _is_exposed(self, node: int) -> bool:
        """Return a boolean representing if the given `node` is exposed.
            node:int, the node to check for.
        """
        return not self._is_in_matching(node)

    def _is_in_matching(self, node: int) -> bool:
        """Return a boolean representing if the given node is in the matching.
            node:int, the node to check for.
        """
        for a, b in self:
            if node == a or node == b:
                return True
        return False

    def _get_match(self, node: int) -> int:
        """Return the match for a given node.
            node:int, the node to retrieve a matching for.
        """
        for a, b in self:
            if a == node:
                return b
            if b == node:
                return a
        raise ValueError("Given node {node} has no match.".format(node=node))

    def _is_even(self, node: int) -> bool:
        """Return a boolean representing if the given node is labeled as `EVEN`.
            node:int, the node to check parity for.
        """
        return self._labels[node]

    def _is_odd(self, node: int) -> bool:
        """Return a boolean representing if the given node is labeled as `ODD`.
            node:int, the node to check parity for.
        """
        return not self._is_even(node)

    def _is_unlabeled(self, node: int) -> bool:
        """Return a boolean representing if the given node isn't labeled.
            node:int, the node to check labeling for.
        """
        return node not in self._labels

    def _set_node_parity(self, node: int, parity: bool):
        """Set a given node with a given parity label.
            node:int, the node to set to a given parity.
            parity:bool, the value of parity to set it to.
        """
        self._labels[node] = parity

    def _set_even(self, node: int):
        """Set a given node to `EVEN`.
            node:int, the node to set to `EVEN`
        """
        self._set_node_parity(node, True)

    def _set_odd(self, node: int):
        """Set a given node to `ODD`.
            node:int, the node to set to `ODD`
        """
        self._set_node_parity(node, False)

    def _prepend_node(self, node: int):
        """Prepend given `node` to `nodes` list.
            node:int, the node to add to list.
        """
        self._nodes.insert(0, node)

    def _pop_node(self) -> int:
        """Return first node in nodes list."""
        return self._nodes.pop()

    def _add_step(self, a: int, b: int):
        """Add node to augmenting path."""
        self._path.append((a, b))
        self._partial_paths.append(self._path)

    def _reset(self):
        """Reset `labels` dictionary and `nodes` list to a clean state."""
        self._labels, self._nodes, self._path, self._partial_paths = {}, [], [], []

    def _search(self, current: int) -> int:
        """Return, if found, the starting node of an augmenting path ending in given `current` node. Otherwise `-1`.
            current:int, the node from which the search will start.
        """
        self._reset()
        self._set_even(current)
        self._prepend_node(current)
        while self._nodes:
            node = self._pop_node()
            if self._is_even(node):
                if self._search_even(node):
                    return True
            else:
                self._search_odd(node)
        return False

    def _search_even(self, vertex: int):
        """Return, if found, the end of the augmenting path otherwise determines the even odd in the path.
            vertex:int, the even node in the current augmenting path.
        """
        for node in self._G.neighbors(vertex):
            if self._is_exposed(node):
                self._add_step(vertex, node)
                return True
            if self._is_in_matching(node) and self._is_unlabeled(node):
                self._set_odd(node)
                self._add_step(vertex, node)
                self._prepend_node(node)
        return False

    def _search_odd(self, vertex: int):
        """Determine the odd step in the augmenting path.
            vertex:int, the odd node in the current augmenting path.
        """
        node = self._get_match(vertex)
        if self._is_unlabeled(node):
            self._set_even(node)
            self._add_step(vertex, node)
            self._prepend_node(node)
            
    def _augment_add_step(self, j, k):
        """Method used for description, implementation in GraficalMatching."""
        pass
    
    def _augment_remove_step(self, j, k):
        """Method used for description, implementation in GraficalMatching."""
        pass

In [10]:
class GraficalMatching(Matching):
    def __init__(self, G: nx.Graph):
        super().__init__(G)
        self.counter = 0
        files = glob.glob('../images/matching/*')
        for f in files:
            os.remove(f)

    def run(self):
        self._setup_describe()
        self._describe_start()
        super().run()
        self._describe_result()

    def _setup_describe(self):
        self._pos = nx.spring_layout(self._G, iterations=10000)
        self._gray, self._dark_gray = '#eeeeee', '#999999'
        positions = np.array(list(self._pos.values()))
        self._min_x, self._min_y = np.min(positions, axis=0) - 0.1
        self._max_x, self._max_y = np.max(positions, axis=0) + 0.1

    def _plot_graphs(self, graphs: list, title: str):
        a = Adigraph(
            layout=self._pos,
            style="-",
            row_size=4,
            caption=title,
            edges_color_fallback="gray!80",
            vertices_color_fallback="gray")

        for i, (G, title) in enumerate(graphs):
            a.add_graph(self._original_G, caption=title, vertices_color={
                i: "cyan!90" if self._original_G.nodes[i]["bipartite"] else "red!90"
                for i in G.nodes
            }, edges_color={
                e: "black!90" for e in G.edges
            }, edges_width={
                e: 1 for e in G.edges
            }, vertices_width={
                v: 1 for v in G.nodes
            })
            
        a.save("../chapters/matching/{c}.tex".format(c=self.counter))
        self.counter += 1
        

    def _augment_add_step(self, j, k):
        self._partial_matchings.append(
            (nx.Graph(self), "Adding edge {k}->{j}.".format(j=j, k=k)))

    def _augment_remove_step(self, j, k):
        self._partial_matchings.append(
            (nx.Graph(self), "Removing edge {k}->{j}.".format(j=j, k=k)))

    def _augment(self):
        """Augments matching folloing augmenting path"""
        self._describe_path(self._path[0][0])
        self._partial_matchings = [(nx.Graph(self), "Previous Matching")]
        super()._augment()
        self._plot_graphs(self._partial_matchings, "Updated Matching.")

    def _delete(self, node: int):
        """Remove given node from graph.
            node:int, the node to remove.
        """
        super()._delete(node)
        self._describe_remove_node(node)

    def _describe_result(self):
        self._plot_graphs(
            [(self._original_G, "Initial Graph"), (self._G, "Final Graph"),
             (nx.Graph(self), "Final Matching")], "Completed Algorithm.")

    def _describe_start(self):
        self._plot_graphs([(self._G, "Original graph"),
                           (nx.Graph(), "Initial Matching")],
                          "Starting with empty matching.")

    def _describe_remove_node(self, node: int):
        self._plot_graphs(
            [(self._G, "Current Graph"), (nx.Graph(self), "Current Matching")],
            "Removed node {node} from graph.".format(node=node))

    def _describe_path(self, start_node: int):
        self._plot_graphs(
            [(nx.Graph(edges), "Step {i}".format(i=i, path=edges))
             for i, edges in enumerate(self._partial_paths)],
            "Searching for path starting from node {start_node}.".format(
                start_node=start_node))

In [11]:
# Create a random graph
seed = 66
random.seed(seed)
np.random.seed(seed)
S, T, edge_probability = 4, 4, 0.5
G = nx.bipartite.random_graph(S, T, edge_probability)

In [12]:
M = GraficalMatching(G)
M.run()

In [14]:
with open("kruskal.tex", "w") as f:
    f.write("".join([open("../chapters/kruskal/"+t).read() for t in sorted(os.listdir("../chapters/kruskal")) if t.endswith(".tex")]))
    